## 18~19년도 대기환경(오존, 미세먼지, 초미세먼지)

In [1]:
import pandas as pd
import numpy as np

- 에어코리아에서 받은 18~19년 데이터를 불러온 후, 하나의 데이터 프레임으로 병합합니다.

In [ ]:
dust19_1 = pd.read_excel('../data/air_19_1.xlsx')
dust19_2 = pd.read_excel('../data/air_19_2.xlsx')
dust19_3 = pd.read_excel('../data/air_19_3.xlsx')
dust19_4 = pd.read_excel('../data/air_19_4.xlsx')
dust19_5 = pd.read_excel('../data/air_19_5.xlsx')
dust19_6 = pd.read_excel('../data/air_19_6.xlsx')
dust19_7 = pd.read_excel('../data/air_19_7.xlsx')
dust19_8 = pd.read_excel('../data/air_19_8.xlsx')
dust19_9 = pd.read_excel('../data/air_19_9.xlsx')
dust19_10 = pd.read_excel('../data/air_19_10.xlsx')
dust19_11 = pd.read_excel('../data/air_19_11.xlsx')
dust19_12 = pd.read_excel('../data/air_19_12.xlsx')

In [39]:
dust18_1 = pd.read_excel('../data/air_18_1.xlsx')
dust18_2 = pd.read_excel('../data/air_18_2.xlsx')
dust18_3 = pd.read_excel('../data/air_18_3.xlsx')
dust18_4 = pd.read_excel('../data/air_18_4.xlsx')

In [41]:
df = pd.concat([dust19_1,dust19_2,dust19_3,dust19_4,
               dust19_5,dust19_6,dust19_7,dust19_8,
               dust19_9,dust19_10,dust19_11,dust19_12,
               dust18_1,dust18_2,dust18_3,dust18_4], axis=0)

- 날짜에 따른 시도별(특별시, 광역시, 도) 대기환경을 구합니다.

In [42]:
def dust_mean(df):
    df['시도'] = df.지역.map(lambda x:x.split()[0])
    df['측정일'] = df.측정일시 // 100
    return df[['시도','측정일','O3','PM10','PM25']].groupby(['시도','측정일']).mean()

In [43]:
df_mean = dust_mean(df)

In [45]:
df_mean = df_mean.reset_index()

In [76]:
df['시군구']=df.지역.map(lambda x:x.split()[1])

- 시도별 대기환경 중, 대한민국 인구 수 8위 내에 드는 서울, 부산, 인천, 대구, 대전, 광주, 울산만을 추출합니다.

In [102]:
condition = (df_mean.시도 == '서울')|(df_mean.시도 == '부산')|(df_mean.시도 == '인천')|(df_mean.시도 == '대구')|(df_mean.시도 == '대전')|(df_mean.시도 == '광주')|(df_mean.시도 == '울산')

df_mean[condition]

,시도,측정일,O3,PM10,PM25
2920,광주,20180101,0.020509,50.337963,18.763889
2921,광주,20180102,0.010901,71.245192,39.188406
2922,광주,20180103,0.021578,29.644550,14.437811
2923,광주,20180104,0.018392,33.753488,18.446512
2924,광주,20180105,0.017606,40.827907,26.046512
...,...,...,...,...,...
8755,인천,20191227,0.023184,31.929467,19.973354
8756,인천,20191228,0.012289,41.895062,21.630841
8757,인천,20191229,0.010069,45.524691,25.029321
8758,인천,20191230,0.018284,33.023112,20.793651


- 수원시 또한 인구 8위 내의 도시입니다. 수원시의 대기환경 데이터를 추출합니다. 

In [95]:
df_suwon = df[df.시군구=='수원시'][['시도','시군구','측정일','O3','PM10','PM25']].groupby(['시도','시군구','측정일']).mean().reset_index()

In [96]:
df_suwon.시도='수원'
del df_suwon['시군구']

- 앞서 추출한 인구 8위 순위 내의 모든 도시의 대기환경 데이터를 병합합니다.

In [3]:
df_top8 = pd.concat([df_suwon,df_mean[condition]], axis=0)

- 변수명을 정리하고, 날짜 데이터의 경우 분석에 용이하도록 dtype을 datetime으로 변환하겠습니다.

In [16]:
df_top8.columns = ['지역', '날짜', '오존(O3)', '미세먼지(PM10)', '초미세먼지(PM2.5)']

In [23]:
df_top8.날짜 = df_top8.날짜.astype('str').map(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])

In [25]:
df_top8.날짜 = pd.to_datetime(df_top8.날짜)

In [31]:
df_top8

,지역,날짜,오존(O3),미세먼지(PM10),초미세먼지(PM2.5)
0,수원,2018-01-01,0.008958,42.766234,21.847561
1,수원,2018-01-02,0.007590,41.896970,23.930818
2,수원,2018-01-03,0.010512,34.670886,18.630952
3,수원,2018-01-04,0.005339,45.394558,26.634146
4,수원,2018-01-05,0.008345,54.585987,37.297468
...,...,...,...,...,...
5835,인천,2019-12-27,0.023184,31.929467,19.973354
5836,인천,2019-12-28,0.012289,41.895062,21.630841
5837,인천,2019-12-29,0.010069,45.524691,25.029321
5838,인천,2019-12-30,0.018284,33.023112,20.793651


- 데이터를 저장합니다.

In [33]:
df_top8.to_csv('../data/air_top8(18~19).csv', encoding='CP949',index=False)